In [ ]:
import tensorflow as tf 
import tensorflow_addons as tfa 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


In [ ]:
train_img_dir = '../input/siim-isic-melanoma-classification/jpeg/train/'
test_img_dir = '../input/siim-isic-melanoma-classification/jpeg/test/'
train = pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')
test = pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')

In [ ]:
# check for balance
train.target.value_counts()

In [ ]:
# trial to balance the dataset manually

negative = train[train['target']==0].sample(2500)
positive = train[train['target']==1]
train=pd.concat([negative,positive])
train=train.reset_index()

In [ ]:
# loading the data
train_car = []
for image_name in train.image_name:
    train_car.append(tf.keras.preprocessing.image.img_to_array(
        tf.keras.preprocessing.image.load_img(path = (train_img_dir + image_name + ".jpg"), color_mode = "rgba", target_size = (128,128)), dtype="float32")
                    )
    

In [ ]:
train["img"] = train_car
test["path"] = [(test_img_dir + img_name + ".jpg") for img_name in test.image_name]

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
# split the data
x_train, x_cross_validation, y_train, y_cross_validation = train_test_split(train.img, train.target, test_size=0.2, shuffle = True, random_state=42)

In [ ]:
# scale the pixels between 0 and 1
x_train /= 255
x_cross_validation /= 255
# redefine dtypes
y_train = np.array(y_train, dtype = "float32")
y_cross_validation = np.array(y_cross_validation, dtype = "float32")
x_train = np.array([np.array(val) for val in x_train])
x_cross_validation = np.array([np.array(val) for val in x_cross_validation])

In [ ]:
model = tf.keras.models.Sequential()

# Convolutional & Max Pooling layers

model.add(tf.keras.layers.Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=(128,128,4)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

# Flatten & Dense layers

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512, activation='relu'))

# performing binary classification
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss = tfa.losses.SigmoidFocalCrossEntropy(),
              optimizer = tf.keras.optimizers.Adam(),
              metrics = ['binary_accuracy',
                       tf.keras.metrics.FalsePositives(),
                       tf.keras.metrics.FalseNegatives(), 
                       tf.keras.metrics.TruePositives(),
                       tf.keras.metrics.TrueNegatives()
                      ]
             )

model.summary()


In [ ]:

# saving the best model for our predictions
checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath="weights.hdf5", verbose=1, save_best_only=True)

history = model.fit(x = x_train,
                    y = y_train,
                    validation_data=(x_cross_validation, y_cross_validation),
                    batch_size=100,
                    epochs=100,
                    verbose=1, 
                    callbacks=[checkpointer]
                   )

In [ ]:
# visualize the loss & accuracy with the train and crossvalidation
f, (f1, f2) = plt.subplots(1, 2, figsize=(12, 6))
f.suptitle('CNN Performance', fontsize=16)

f1.plot(history.history['binary_accuracy'], label='Train Accuracy')
f1.plot(history.history['val_binary_accuracy'], label='Cross Validation Accuracy')
f1.set_ylabel('Accuracy')
f1.set_xlabel('Epoch')
f1.set_title('Accuracy')
f1.legend(loc="best")

f2.plot(history.history['loss'], label='Train Loss')
f2.plot(history.history['val_loss'], label='Cross Validation Loss')
f2.set_ylabel('Loss')
f2.set_xlabel('Epoch')
f2.set_title('Loss')
f2.legend(loc="best")

In [ ]:
# input to be predicted
test = tf.keras.preprocessing.image.img_to_array(tf.keras.preprocessing.image.load_img(path = ("../input/siim-isic-melanoma-classification/jpeg/test/ISIC_0052060.jpg"), color_mode = "rgba", target_size = (128,128)), dtype="float32")
test = test / 255
test=np.reshape(test,(1,128,128,4))
test = np.array(test)
# load the best model
model.load_weights('weights.hdf5')

model.predict(test)
    
